In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

from TropiGAT_functions import *
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_ensemble = f"{path_work}/ficheros_28032023/ensemble_1908"


> Ferrol

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)

> Bea

In [4]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

bea_embeddings = pd.read_csv(f"{path_project}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings.set_index([0], inplace = True)
bea_embeddings


,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,,,,,,,,,,,,,,,,,,,,,
A1c_00034,-0.031228,0.018990,0.011914,-0.036915,-0.085279,0.074510,0.038491,-0.092385,0.041953,0.006984,...,0.024149,-0.001948,0.086334,0.015169,-0.009941,0.070748,0.061202,-0.088183,0.029006,0.092003
A1h_00013,0.003407,0.058613,0.004211,-0.032270,-0.080894,0.003224,0.026575,-0.049651,0.068412,0.048845,...,-0.037309,-0.002876,0.103554,-0.011835,0.040593,0.037963,0.031057,-0.102158,0.077059,0.088456
S11a_00036,-0.000087,0.046707,0.027543,-0.037515,-0.073041,0.012802,0.033908,-0.055600,0.117382,0.031282,...,-0.037399,0.023287,0.144737,-0.022651,0.048932,0.046084,0.025735,-0.100239,0.049737,0.135533
A1q_00023,0.035220,-0.004737,-0.047403,0.058447,-0.127027,0.047977,0.093704,-0.168074,0.073823,-0.002389,...,0.015234,0.019885,0.038854,0.031011,-0.032295,-0.027215,0.027464,-0.042393,-0.133780,0.124112
A1q_00010,0.006108,0.036785,0.033718,-0.061564,-0.102759,0.008589,0.025191,-0.038584,0.046164,0.036677,...,-0.030083,-0.019619,0.079121,-0.012699,0.048490,0.054487,0.050371,-0.102361,0.034225,0.098482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A1e_00012,0.023357,0.039636,-0.014646,-0.014371,-0.082256,0.020024,-0.017925,-0.063363,0.068452,0.040234,...,0.016703,-0.005695,0.106709,-0.003896,0.034324,0.069450,0.058052,-0.072267,0.086228,0.071590
M5a_00170,-0.021793,0.077967,-0.017637,-0.069029,-0.085250,0.012727,0.054289,-0.040869,0.069873,0.026029,...,-0.023246,-0.027288,0.092628,-0.031385,0.033944,0.073543,0.025255,-0.100661,0.049953,0.117564
A1j_00002,0.036885,-0.005403,-0.047889,0.057499,-0.128569,0.041746,0.098268,-0.166326,0.072257,-0.004946,...,0.011941,0.016783,0.038163,0.026237,-0.031345,-0.027221,0.030271,-0.040723,-0.132024,0.123907


> Towndsend

In [5]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

towndsend_embeddings = pd.read_csv(f"{path_project}/Townsed_phages.esm2.embedding.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings.set_index([0], inplace = True)

towndsend_embeddings

,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,,,,,,,,,,,,,,,,,,,,,
PP187_gp233,0.016811,0.073770,-0.009818,-0.036033,-0.085370,-0.016310,0.007661,-0.083562,0.072318,0.050186,...,-0.004649,-0.006715,0.121524,-0.004321,0.019053,0.074084,0.042733,-0.064831,0.040088,0.062354
LCALLHIG_00050,0.015476,0.014564,0.016806,-0.042899,-0.071834,0.021123,0.059287,-0.088285,0.081127,0.022274,...,0.016373,-0.006337,0.073145,-0.024415,0.028782,0.048048,0.012546,-0.085356,-0.006697,0.106924
NBNDMPCG_00165,0.097851,-0.005817,0.044202,0.066516,-0.062639,0.012461,-0.056078,-0.046124,0.025547,0.090553,...,0.091665,0.004563,0.011115,0.032996,0.009146,0.084371,0.129982,-0.089394,-0.080854,-0.026717
EKPIEFBL_00177,0.062666,-0.010341,-0.000893,0.034539,-0.107698,0.021799,-0.030059,-0.039033,-0.016901,0.012905,...,0.085217,-0.028300,0.048211,0.066592,0.019395,0.056614,0.101654,-0.031980,-0.056580,0.012406
NBNDMPCG_00161,0.003200,0.046110,0.004314,-0.063110,-0.094585,-0.018906,-0.011139,-0.041858,0.066822,0.052831,...,-0.010727,-0.004597,0.068099,-0.013416,0.020944,0.050755,0.065946,-0.076776,0.066887,0.054853
NBNDMPCG_00168,0.001259,0.060305,0.032569,-0.041890,-0.089129,-0.013875,0.007752,-0.087015,0.005353,0.029473,...,0.015444,-0.034203,0.108186,-0.001604,0.013760,0.058938,0.048440,-0.139902,0.032722,0.077596
PP187_gp235,-0.002062,0.030683,0.003746,-0.050332,-0.096769,-0.008238,-0.007483,-0.037646,0.022372,0.045324,...,0.000123,0.011784,0.091126,-0.005352,0.009611,0.032730,0.073179,-0.076450,0.046952,0.091653
PP187_gp225,0.056159,0.023400,0.017573,0.018860,-0.138268,0.037129,-0.049114,-0.078060,0.031631,0.034067,...,0.026249,-0.017010,0.071993,0.054659,0.010754,0.058265,0.042473,-0.050659,0.040769,0.031806
EONHMLJF_00087,0.040388,0.038805,0.016201,0.074880,-0.099272,0.002561,0.038002,-0.066288,0.094638,0.019382,...,0.003874,-0.040858,0.037478,0.061814,0.134470,-0.062697,0.019345,-0.057822,0.080840,0.092654


***
> run the predictions 

In [6]:
# Run the predictions Bea :
bea_graph = graph_single_Dpo_pred(bea_embeddings)
bea_pred = run_predictions(bea_graph,bea_embeddings)
bea_pred_formated = format_predictions(bea_pred , sep = "_")


In [7]:
bea_pred_clean = get_top_n_kltypes(bea_pred_formated , 15)
clean_print(bea_pred_clean)

{'A1a': [('KL70', 5.320959), ('KL55', 5.306431), ('KL170', 5.02337), ('KL67', 4.443937), ('KL151', 4.317976), ('KL7', 4.2932773),
         ('KL124', 4.2669067), ('KL35', 4.2052803), ('KL119', 4.2025247), ('KL82', 4.0291615), ('KL61', 3.6921368), ('KL169', 3.6910706),
         ('KL4', 3.5605159), ('KL41', 3.511106), ('KL62', 3.4014263)],
 'A1b': [('KL41', 5.1059866), ('KL7', 4.812174), ('KL149', 4.07627), ('KL151', 3.99238), ('KL170', 3.843959), ('KL61', 3.5482726), ('KL4', 3.5146015),
         ('KL8', 3.5110102), ('KL43', 3.4916542), ('KL62', 3.380776), ('KL70', 3.2794728), ('KL124', 3.164495), ('KL140', 3.0880995),
         ('KL119', 2.9684653), ('KL66', 2.9238212)],
 'A1c': [('KL170', 5.531213), ('KL41', 5.398126), ('KL4', 5.1381397), ('KL7', 4.9298563), ('KL55', 4.766929), ('KL62', 4.640277),
         ('KL151', 4.6137533), ('KL61', 4.5852365), ('KL35', 4.5695267), ('KL43', 4.3121157), ('KL10', 4.291616), ('KL15', 4.115518),
         ('KL102', 4.0913634), ('KL169', 3.8118734), ('KL5'

In [10]:
# Run the predictions Towndsend :
towndsend_graph = graph_single_Dpo_pred(towndsend_embeddings)
towndsend_pred = run_predictions(towndsend_graph,towndsend_embeddings)
towndsend_pred_formated = format_predictions(towndsend_pred , sep = "_")


In [11]:
towndsend_pred_clean = get_top_n_kltypes(towndsend_pred_formated , 15)
clean_print(towndsend_pred_clean)

{'BLCJPOBP': [('KL170', 5.218863), ('KL41', 4.8893113), ('KL55', 4.781771), ('KL7', 4.7510695), ('KL151', 4.5356336), ('KL61', 4.400956),
              ('KL35', 4.36309), ('KL124', 4.3321295), ('KL62', 3.804236), ('KL169', 3.7823238), ('KL4', 3.7436855), ('KL5', 3.580293),
              ('KL81', 3.5710385), ('KL123', 3.299664), ('KL10', 3.2020826)],
 'DIMCIIMF': [('KL4', 3.631734), ('KL123', 3.464705), ('KL27', 3.27533), ('KL41', 3.2331758), ('KL170', 2.9942484), ('KL132', 2.8753114),
              ('KL53', 2.8636796), ('KL127', 2.664638), ('KL145', 2.6059532), ('KL60', 2.5001318), ('KL37', 2.4821396), ('KL143', 2.3671446),
              ('KL159', 2.3105178), ('KL151', 2.2412372), ('KL139', 2.2262592)],
 'DJLANJJD': [('KL170', 2.8457952), ('KL27', 2.4912138), ('KL141', 1.9840983), ('KL29', 1.9575403), ('KL139', 1.8858476), ('KL105', 1.854229),
              ('KL145', 1.7999907), ('KL23', 1.7889607), ('KL123', 1.783452), ('KL53', 1.7589215), ('KL20', 1.7465144), ('KL4', 1.7324631),
    